In [1]:
# https://chromedriver.chromium.org/downloads

In [2]:
from selenium import webdriver
DRIVER_PATH = "C:/Users/Praveen Kumar/Desktop/chromedriver_win32/chromedriver.exe"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [8]:
import os
import time
import io
from PIL import Image
import hashlib
import requests
from bs4 import BeautifulSoup

In [9]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.rg_ic")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.irc_mi')
            for actual_image in actual_images:
                if actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(1)
            load_more_button = wd.find_element_by_css_selector(".ksb")
            if load_more_button:
                wd.execute_script("document.querySelector('.ksb').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [10]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [13]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=150):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [15]:
term = "lamborgini car callpapers"
search_and_download(search_term = term,driver_path = DRIVER_PATH)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, looking for more ...
Found: 300 search results. Extracting links from 100:300
Found: 150 image links, done!
ERROR - Could not save https://hdwallsbox.com/wallpapers/l/1920x1080/65/classic-lamborghini-auto-black-cars-1920x1080-64341.jpg - cannot identify image file <_io.BytesIO object at 0x000001E6305B3AC8>
SUCCESS - saved https://www.setaswall.com/wp-content/uploads/2017/03/Lamborghini-With-Everythin-Open-In-Hail-Wallpaper-1920x1080.jpg - as ./images\lamborgini_car_callpapers\20a82a6bb5.jpg
SUCCESS - saved https://wallpaperstock.net/wallpapers/thumbs1/54833hd.jpg - as ./images\lamborgini_car_callpapers\8ef544eb3b.jpg
SUCCESS - saved https://avante.biz/wp-content/uploads/Lamborghini-Images-Wallpapers/Lamborghini-Images-Wallpapers-057.jpg - as ./images\lamborgini_car_callpapers\c251ff066b.jpg
SUCCESS - saved https://cache.desktopnexus.com/thumbseg/2213/2213452-bigthumbnail.jpg - as ./images\lamborgini_car_call

SUCCESS - saved https://www.jakpost.travel/wimages/large/42-423416_lamborghini-cars-wallpapers-for-desktop.jpg - as ./images\lamborgini_car_callpapers\b7f082ed53.jpg
SUCCESS - saved https://i.pinimg.com/originals/b0/19/25/b0192588f85c50d84c5e57314b1aad1d.jpg - as ./images\lamborgini_car_callpapers\a86f07aebb.jpg
SUCCESS - saved https://www.itl.cat/pngfile/big/164-1640424_hd-car-wallpapers-high-resolution-cars-motor-sports.jpg - as ./images\lamborgini_car_callpapers\5cefc4398a.jpg
SUCCESS - saved https://images.all-free-download.com/images/wallpapers_large/imsa_lamborghini_gallardo_wallpaper_lamborghini_cars_wallpaper_2123.jpg - as ./images\lamborgini_car_callpapers\7ac6e7cf13.jpg
SUCCESS - saved https://r1.ilikewallpaper.net/iphone-8-wallpapers/download/27163/Lamborghini-Car-Dark-Black-Awesome-iphone-8-wallpaper-ilikewallpaper_com.jpg - as ./images\lamborgini_car_callpapers\ca15ca24d3.jpg
SUCCESS - saved http://data.freehdw.com/new-yellow-lamborghini-huracan-2015-hd-car.jpg - as ./imag

SUCCESS - saved https://www.pixelstalk.net/wp-content/uploads/images2/Lamborghini-Wallpaper-Photo.jpg - as ./images\lamborgini_car_callpapers\478f6e1250.jpg
SUCCESS - saved http://files.all-free-download.com//downloadfiles/wallpapers/3840_2160/lamborghini_centenario_hyper_car_17385.jpg - as ./images\lamborgini_car_callpapers\238e482772.jpg
SUCCESS - saved http://www.hdnicewallpapers.com/Walls/Big/Lamborghini/Wonderful_Green_Lamborghini_Car_Wallpaper.jpg - as ./images\lamborgini_car_callpapers\1b5b1163c3.jpg
SUCCESS - saved https://lh6.googleusercontent.com/proxy/rKMI6YSOpRcQScNU12EbdQwlkRmmluhJG22g-9ooDB_vCwriR4xWL2DVQ-X2Ljrnl4Q3UmpbOoSiNk2A3FBJ9Om_l_vdmEAUPpLbYbnxTCI5JQ=s0-d - as ./images\lamborgini_car_callpapers\4fc91783e4.jpg
SUCCESS - saved http://www.hdnicewallpapers.com/Walls/Big/Lamborghini/4K_Car_Wallpaper_of_2019_Lamborghini_SC18_Alston.jpg - as ./images\lamborgini_car_callpapers\93db56ac82.jpg
SUCCESS - saved https://images5.alphacoders.com/982/thumb-350-982211.jpg - as ./im

SUCCESS - saved https://s.itl.cat/pngfile/s/56-560602_hd-car-wallpapers-high-resolution-cars-wheels-motor.jpg - as ./images\lamborgini_car_callpapers\3cedace28b.jpg
SUCCESS - saved https://www.highreshdwallpapers.com/wp-content/uploads/2017/05/Lamborghini-Fast-Car-Wallpaper.jpg - as ./images\lamborgini_car_callpapers\dbe944f562.jpg
SUCCESS - saved https://image.winudf.com/v2/image/Y29tLmdhY2hvdXJwcm8uZ2FjaG91cnByb2xfc2NyZWVuXzJfMTUyMjI3NjQ5MF8wNjg/screen-2.jpg?fakeurl=1&type=.jpg - as ./images\lamborgini_car_callpapers\bc27f600cd.jpg
SUCCESS - saved http://getwallpapers.com/wallpaper/full/c/6/7/937399-large-lamborghini-cars-wallpaper-1920x1080-for-iphone-7.jpg - as ./images\lamborgini_car_callpapers\c2779d47a1.jpg
SUCCESS - saved https://cdn.soft112.com/lamborghini-cars-wallpapers-hd/00/00/0F/W2/00000FW2OP/pad_screenshot.jpg - as ./images\lamborgini_car_callpapers\8c327097fc.jpg
SUCCESS - saved https://www.hd-freewallpapers.com/lamborghini-wallpaper/desktop-lamborghini-cars-photos.jpg 